In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [1]:
cd ..

/home/chiroptera/workspace/QCThesis


In [240]:
import MyML.graph.mst
reload(MyML.graph.mst)
from MyML.graph.mst import boruvkaMinhoSeq, boruvkaMinhoSeqForest

In [249]:
dest2 = np.array([3, 1, 2, 3, 0, 2, 0, 1, 0, 1, 7, 5, 6, 4, 6, 5, 4, 4], dtype = np.int32)
weight2 = np.array([4, 2, 5, 3, 2, 4, 5, 4, 4, 3, 2, 4, 8, 4, 3, 3, 8, 2], dtype = np.float32)
fe2 = np.array([0, 3, 6, 8, 10, 13, 15, 17], dtype = np.int32)
od2 = np.array([3, 3, 2, 2, 3, 2, 2, 1], dtype = np.int32)

In [250]:
mst_solver = boruvkaMinhoSeqForest(dest2, weight2, fe2, od2)

In [243]:
mst_solver.fit()

array([ 4,  7,  9, 15, 17, 13,  3], dtype=int32)

In [190]:
mst_solver = boruvkaMinhoSeq(dest2, weight2, fe2, od2)

In [261]:
print mst_solver.n_components
print mst_solver.n_mst

3
2


In [262]:
#%%debug
mst_solver.findMinEdge()
mst_solver.vertex_minedge

array([-1,  0,  2], dtype=int32)

In [263]:
mst_solver.removeMirrored()
mst_solver.vertex_minedge

array([-1, -1,  2], dtype=int32)

In [265]:
mst_solver.initColors()
print mst_solver.colors

[0 1 1]


In [266]:
while(not mst_solver.converged):
    mst_solver.propagateColors()
print mst_solver.colors

[0 1 1]


In [267]:
mst_solver.createNewVertexID()
print mst_solver.new_vertex

[0 1 2]


In [268]:
mst_solver.countNewVertex()
print mst_solver.new_first_edge
print mst_solver.new_outDegree

[0 0]
[0 0]


In [269]:
mst_solver.removeMST()

In [271]:
mst_solver.n_mst

3

In [276]:
print mst_solver.edge_id
print mst_solver.dest
print mst_solver.weight
print mst_solver.firstEdge
print mst_solver.outDegree

[]
[]
[]
[0 0]
[0 0]


In [273]:
mst_solver.assignInsert()
print mst_solver.edge_id
print mst_solver.dest
print mst_solver.weight
print mst_solver.firstEdge
print mst_solver.outDegree

AttributeError: boruvkaMinhoSeqForest instance has no attribute 'new_first_edge'

In [275]:
mst_solver.n_components

2

In [ ]:
mst_solver.fit()

In [1]:
import numpy as np
from numba import cuda, int32

In [8]:
a = np.arange(2**4, dtype=np.int32)
dA = cuda.to_device(a)

In [1]:
import numpy as np
import numba
from numba import cuda
@cuda.jit
def smallScanSM(x):
    temp = cuda.shared.array(0,dtype=numba.int32)
    thid = cuda.grid(1)
    #thid = cuda.threadIdx.x

    # copy a to shared memory with a shift right
    if thid > 0:
        temp[thid] = x[thid-1]
    else:
        temp[thid] = 0
        
    cuda.syncthreads()
    
    d = 1
    while(d < x.size):
        if thid >= d:
            val_a = temp[thid]
            val_b = temp[thid - d]
            temp[thid] = val_a + val_b
        d *= 2
        cuda.syncthreads()
    
    # copy results back    
    cuda.syncthreads()
    x[thid] = temp[thid]
    
a = np.arange(2**4, dtype=np.int32)

stream = 0
block_count = 1
thread_count = a.size
dyn_shared_size = a.size * a.itemsize
dA = cuda.to_device(a)
smallScanSM[block_count,thread_count,stream,dyn_shared_size](dA)
a = dA.copy_to_host()

In [2]:
a = dA.copy_to_host()

In [3]:
a

array([  0,   0,   1,   3,   6,  10,  15,  21,  28,  36,  45,  55,  66,
        78,  91, 105], dtype=int32)

In [1]:
cd ~/QCThesis

/home/chiroptera/workspace/QCThesis


In [2]:
import numpy as np
from MyML.cluster.K_Means3 import K_Means, _cu_label_kernel_dists
from MyML.cluster.eac import EAC
from MyML.metrics import accuracy
from MyML.helper.partition import generateEnsemble
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans
from numbapro import cuda

In [8]:
n_samples = 1e4
n_features = 2
centers = 6
n_samples = np.int(n_samples)
data, gt = make_blobs(n_samples = n_samples, n_features = n_features, centers = centers)
data = data.astype(np.float32)
n_samples_sqrt = np.int(np.sqrt(n_samples))
n_clusters = [n_samples_sqrt / 2, n_samples_sqrt]
n_clusters = map(int,n_clusters)

In [9]:
generator = K_Means(cuda_mem="manual")

In [10]:
generator.n_clusters = n_clusters[-1]
generator.max_iter = 1

In [11]:
generator.N = n_samples
generator.D = n_features

generator.centroids = generator._init_centroids(data)
generator.labels = cuda.pinned_array(shape=generator.N, dtype = np.int32)
generator._dists = cuda.pinned_array(shape=generator.N, dtype = np.float32)

generator._compute_cuda_dims(data)
gridDim = generator._gridDim
blockDim = generator._blockDim

dData = cuda.to_device(data)

dCentroids = cuda.to_device(generator.centroids)

dLabels = cuda.device_array_like(generator.labels)
dDists = cuda.device_array_like(generator._dists)



CudaAPIError: Call to cuMemFreeHost results in CUDA_ERROR_LAUNCH_TIMEOUT

In [7]:
startE = cuda.event()
endE = cuda.event()

startE.record()
_cu_label_kernel_dists[gridDim,blockDim](dData,dCentroids,dLabels,dDists)
endE.record()
endE.synchronize()
print cuda.event_elapsed_time(startE,endE)

startE.record()
dDists.copy_to_host(ary = generator._dists)
labels = dLabels.copy_to_host(ary = generator.labels)
endE.record()
endE.synchronize()
print cuda.event_elapsed_time(startE,endE)

CudaAPIError: Call to cuEventSynchronize results in CUDA_ERROR_LAUNCH_TIMEOUT

In [8]:
%time generator.fit(data)

CudaAPIError: Call to cuMemcpyDtoH results in CUDA_ERROR_LAUNCH_TIMEOUT

In [21]:
del generator

NameError: name 'generator' is not defined

In [6]:
from numbapro import cuda

In [21]:
a=cuda.to_device(data)

In [24]:
a=cuda.device_array(10000)

In [28]:
10000/1000

10

In [25]:
b=a.split(1000)

In [26]:
c=b.next()

In [27]:
b.

8000